#### Async/await
##### Полезные ссылки
  * https://realpython.com/async-io-python/
  
##### Введение и мотивация
Существуют два типа программ по характеру нагрузки на систему.CPU-bound и IO-bound программы  Вот основные различия:

**CPU-bound программы**

Определение: Программы, которые большую часть времени тратят на вычисления
Характеристики:
  * Высокая загрузка процессора (CPU utilization близок к 100%)
  * Минимальное время ожидания
  * Производительность зависит от мощности CPU

Примеры:
  * Математические вычисления
  * Шифрование/дешифрование
  * Обработка видео/аудио
  * Сортировка больших массивов

**IO-bound программы**
Определение: Программы, которые часто ожидают завершения операций ввода-вывода
Характеристики:
  * Низкая загрузка CPU
  * Частые простои при ожидании диска/сети
  * Производительность зависит от скорости IO

Примеры:
  * Чтение/запись файлов
  * Сетевые запросы
  * Работа с базами данных
  * Пользовательский ввод

Рассмотрим Python и IO-bound программы c параллельной обработкой нескольких запросов

#### GIL (Global Interpreter Lock)
Потоков в python программе может быть много, но одновременно исполняется только один, вне зависимости от количества ядер в процессоре и пользовательской бизнес-логики. Работает это за счет того, что исполняющий поток захватывает GIL, и другие потоки не могут исполнять код, пока GIL не освобожден.
Но если поток уходи в операции ввода/вывода, то он освобождает GIL, и другой поток может исполнять код.

#### Подходы к параллельной обработке
**Последовательная обработка**

```python
for request in incoming_requests:
    process_request(request):
    
```
Это не масштабируемо, потому что одновременно может обрабатываться только один запрос, и они будут в очереди копиться быстрее, чем обрабатываться. Здесь хороший пример - сеанс (не) одновременной игры гроссмейстера с любителями, если гроссмейстер будет играть с ними по-очереди, то матч безбожно затянется

**Параллельная обработка**

```python
import threading

def process_request(request):
    print(request)
    
threads = []
for request in incoming_requests:
    t = threading.Thread(target=process_request, args=(request,)
    threads.append(t)
    t.start()
```
Потоки - дорогая конструкция (сравнимая с процессом), их не может быть сколь угодно, упремся в ресурсы системы, + с ростом количества потоков растут накладные расходы на постоянное переключение между ними. Кроме того, из-за GIL в python они работают крайне неээфективно

**Асинхронная обработка**

```python
import asyncio

async def process_request(request):
    print(request)
    
async def main():
    tasks = []
    for request in incoming_requests:
        tasks.append(asyncio.create_task(process_request(request)))
    await asyncio.gather(*tasks)
  

asyncio.run(main())
```
В основе этого механизма лежит `event loop`: подход, когда программа в цикле определяет, не пора ли инициировать какую-то новую неблокирующую операцию и не пора ли сделать что-то с результатом завершенной операции, инициированной ранее. Сами операции не блокируют основной поток исполнения программы.

**Конструкции python для асинхронной обработки**
`async/await` - ключевые слова.
* `async` - объявляет функцию как асинхронную
* `await` - ожидает завершения асинхронной операции

`asyncio` - модуль для работы с асинхронными операциями.

**Асинхронная функция**
```python
async def process_request(request):
    # здесь обычный код, с небольшой особенностью, нельзя использовать `yield from`. Не очень и хотелось
    print(request)
    # await - ожидает завершения асинхронной операции. Это сообщение системе, что пока эта операция не завершена, нет смысла продолжать, и можно переключиться на другую задачу
    await asyncio.sleep(1)
    return request

```


In [24]:
import asyncio

async def process_request(request):
    print(request)
    return f"Processed {request}"

#process_request("aaa")

async def main():
    result = await process_request("aaa")


    return result

if __name__ == "__main__":
    result = await main()
    print(result)
    # стоило бы использовать asyncio.run(main()), но он не работает в нотбуках


aaa
Processed aaa


In [28]:
import asyncio
import aiohttp

async def check(url):
    async with aiohttp.ClientSession() as session:
         async with session.get(url) as response:
             print(f"{url}: status -> {response.status}")


async def main():
    websites = [
        "https://realpython.com",
         "https://pycoders.com",
         "https://www.python.org",
         "http://yandex.ru/nothingfound"
    ]
    # asyncio.gather(*tasks) дожидается завершения всех задач, которые исполняются асинхронно, параллельно и независимо
    await asyncio.gather(*(check(url) for url in websites))


await main()

http://yandex.ru/nothingfound: status -> 404
https://www.python.org: status -> 200
https://pycoders.com: status -> 200
https://realpython.com: status -> 200


In [30]:
import asyncio
import time

async def sync_sleep():
    time.sleep(1)

async  def async_sleep():
    await asyncio.sleep(1)

async def main():
    start = time.perf_counter()
    await asyncio.gather(
        # поскольку sync_sleep полностью синхронен, никаких await внутри нет, все 3 вызова исполняются последовательно
        # никакого выиигрыша по времени нет
        *(sync_sleep() for _ in range(10))
    )
    end = time.perf_counter()
    print(f"\n==> Total time: {end - start:.2f} seconds")

    await asyncio.gather(
        # а здесь, поскольку внутри есть await, обработчик event loop может начать обработку следующей задачи, и выигрыш по времени есть
        *(async_sleep() for _ in range(10))
    )
    end = time.perf_counter()
    print(f"\n==> Total time: {end - start:.2f} seconds")

if __name__ == "__main__":
    # asyncio.run(main())
    await main()


==> Total time: 10.03 seconds

==> Total time: 11.03 seconds
